## Data Cleaning

In [2]:
# import needed libraries
import pandas as pd 

# Local imports
# from source.constants import PUNCT_TO_REMOVE
import sys
import os
target = os.path.abspath(os.path.abspath(''))
while(target.split("\\")[-1]!="Road"):
    target = os.path.dirname(target)
sys.path.append(target)
from source.features import data_cleaning as DC

In [3]:
# read data
data = pd.read_csv("../data/interim/feature_selected.csv")

In [4]:
data.shape

(17306, 10)

In [5]:
data.head()

,id,message,from_id,reply_to,year,month,day,hour,minute,second
0,409414,.,5774169366,-1,2023,3,7,13,48,25
1,409427,حواره للخارج من نابلس ؟,325418964,-1,2023,3,7,13,50,2
2,409428,سالك سالك,1827882971,409427,2023,3,7,13,50,9
3,409429,حوارة في عليها مستوطنين,5633332941,-1,2023,3,7,13,50,18
4,409430,وين,1332172310,409429,2023,3,7,13,50,26


### Apply the cleaning process to the messages

In [6]:
# apply the clean_data function to the 'message' column using apply
data['message'] = data.apply(DC.clean_data, axis=1)

# drop any rows where the 'message' column is empty
data = data.dropna(subset=['message'])

In [7]:
data.head()

,id,message,from_id,reply_to,year,month,day,hour,minute,second
1,409427,حواره للخارج من نابلس ؟,325418964,-1,2023,3,7,13,50,2
2,409428,سالك سالك,1827882971,409427,2023,3,7,13,50,9
3,409429,حوارة في عليها مستوطنين,5633332941,-1,2023,3,7,13,50,18
4,409430,وين,1332172310,409429,2023,3,7,13,50,26
5,409431,كيف طريق من طوباس لرام الله,5661074759,-1,2023,3,7,13,50,35


In [8]:
data.shape

(17152, 10)

### Map each message to its corresponding reply and delete any replies found.


In [9]:
data = DC.map_reply_messages(data)

In [10]:
data.head()

,id,message,from_id,reply_to,year,month,day,hour,minute,second,reply
1,409427,حواره للخارج من نابلس ؟,325418964,-1,2023,3,7,13,50,2,سالك سالك
2,409428,سالك سالك,1827882971,409427,2023,3,7,13,50,9,NaN
3,409429,حوارة في عليها مستوطنين,5633332941,-1,2023,3,7,13,50,18,لا مافب
4,409430,وين,1332172310,409429,2023,3,7,13,50,26,NaN
5,409431,كيف طريق من طوباس لرام الله,5661074759,-1,2023,3,7,13,50,35,NaN


In [11]:
data.shape

(17152, 11)

In [12]:
data.drop(columns="reply_to", axis=1, inplace=True)

In [13]:
data.to_csv("../data/interim/cleaned_data.csv")